# Load Packages

In [1]:
import warnings
warnings.filterwarnings('ignore')

try:
    from ibm_aigov_facts_client import AIGovFactsClient
except:
    !pip install -U ibm-aigov-facts-client
    from ibm_aigov_facts_client import AIGovFactsClient

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.0/140.0 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 29.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 115.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.6/547.6 kB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 kB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 58.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 45.9 MB/s eta 0:00:00
  Created wheel for timeout-decorator: filename=timeout_decorator-0.5.0-py3-none-any.whl size=5006 sha256=4ac480f64b541d3a6b79d6f41f5ad883c121cbd35a8676121575b614e6070372
  Stored in directory: /tmp/1000650000/.cache/pip/wheels/68/2f/bc/76f1192d474666d41ae6f09813fccbd00fe3f07e8261c4cff5
  Created wheel for databricks-cli: filename=databricks_cli-0.17

In [2]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3
from project_lib import Project

#sklearn 
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score
import os, types
from sklearn.metrics import f1_score
import numpy as np
import joblib
import datetime
import tempfile

#WML
from ibm_watson_machine_learning import APIClient

#AiGovernance
from ibm_aigov_facts_client import AIGovFactsClient,CloudPakforDataConfig
import sys

# Load Data

In [5]:

import itc_utils.flight_service as itcfs
readClient = itcfs.get_flight_client()

nb_data_request = {
    'data_name': """modeling_records_2022.csv""",
    'interaction_properties': {
        #'row_limit': 500,
        'infer_schema': 'true',
        'infer_as_varchar': 'false'
    }
}

flightInfo = itcfs.get_flight_info(readClient, nb_data_request=nb_data_request)

df = itcfs.read_pandas_and_concat(readClient, flightInfo, timeout=240)
df.head(10)

,POSITION_CODE,DEPARTMENT_CODE,DAYS_WITH_COMPANY,COMMUTE_TIME,AGE_BEGIN_PERIOD,GENDER_CODE,ATTRITION,PERIOD_TOTAL_DAYS,STARTING_SALARY,ENDING_SALARY,...,VACATION_DAYS_TAKEN,SICK_DAYS_TAKEN,PROMOTIONS,NB_MANAGERS,DAYS_IN_POSITION,DAYS_SINCE_LAST_RAISE,RANKING_CODE,OVERTIME,DBLOVERTIME,TRAVEL
0,1200,200,1825,29,55,False,False,330,159230.77,161538.46,...,28,10.5,False,1,1825,0,3,0.0,0.0,False
1,1200,200,2615,0,49,False,False,180,181153.85,183846.15,...,15,4.0,False,1,2615,60,3,0.0,0.0,False
2,1300,320,1609,30,44,True,False,330,129692.31,132923.08,...,20,15.0,False,1,1609,150,3,0.0,0.0,False
3,1300,320,2035,13,45,False,False,330,146769.23,150461.54,...,28,8.0,False,1,2035,210,3,0.0,0.0,False
4,1400,380,1885,31,44,False,False,330,146769.23,150461.54,...,26,11.5,False,1,1885,60,3,0.0,0.0,False
5,1400,380,3070,35,40,False,False,330,157846.15,161538.46,...,19,12.5,False,1,3070,150,3,0.0,0.0,False
6,1400,380,1825,9,40,True,False,330,146769.23,150461.54,...,20,9.0,False,1,1825,0,3,0.0,0.0,False
7,1500,300,2035,11,46,False,False,330,132923.08,136153.85,...,10,9.0,False,1,2035,210,3,0.0,0.0,False
8,1500,300,2250,0,45,True,False,180,150461.54,154153.85,...,10,4.0,False,1,2250,60,3,0.0,0.0,False
9,1600,330,1520,31,46,False,False,330,143076.92,146769.23,...,26,9.5,False,1,1520,60,3,0.0,0.0,False


In [6]:
from sklearn.model_selection import train_test_split

X = df.drop(['ATTRITION'], axis=1)  # Features
y = df['ATTRITION']  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15) # 85% training and 15% test

In [7]:
X.columns.tolist()

['POSITION_CODE',
 'DEPARTMENT_CODE',
 'DAYS_WITH_COMPANY',
 'COMMUTE_TIME',
 'AGE_BEGIN_PERIOD',
 'GENDER_CODE',
 'PERIOD_TOTAL_DAYS',
 'STARTING_SALARY',
 'ENDING_SALARY',
 'NB_INCREASES',
 'BONUS',
 'NB_BONUS',
 'VACATION_DAYS_TAKEN',
 'SICK_DAYS_TAKEN',
 'PROMOTIONS',
 'NB_MANAGERS',
 'DAYS_IN_POSITION',
 'DAYS_SINCE_LAST_RAISE',
 'RANKING_CODE',
 'OVERTIME',
 'DBLOVERTIME',
 'TRAVEL']

The cell below tells the Watson Machine Learning client to save the models in the current project. If you receive an error here, it is likely because you did not correctly set your project ID at the beginning of the notebook.

The following cell provides connection information to the model training data, which will be stored with the model and in FactSheets. You could use the Cloud Object Storage information for this particular project by changing the credentials to match those from above where you inserted the file to code, but for simplicity's sake, you will use a pre-existing file.

In [8]:
training_data_references = [
                {
                    "id": "attrition",
                    "type": "s3",
                    "connection": {
                        "access_key_id": "yqcPbWZ0AQPHleHVerrR4Wx5e9pymBdMgydbEra5zCif",
                        "endpoint_url": "https://s3.us.cloud-object-storage.appdomain.cloud",
                        "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/7d8b3c34272c0980d973d3e40be9e9d2:2883ef10-23f1-4592-8582-2f2ef4973639::"
                    },
                    "location": {
                        "bucket": "faststartlab-donotdelete-pr-nhfd4jnhlxgpc7",
                        "path": "modeling_records_2022.csv"
                    },
                    "schema": {
                        "id": "training_schema",
                        "fields": [
                            {"name": "POSITION_CODE", "nullable": True, "metadata": {}, "type": "double"},
                            {"name": "DEPARTMENT_CODE", "nullable": True, "metadata": {}, "type": "double"},
                            {"name": "DAYS_WITH_COMPANY", "nullable": True, "metadata": {}, "type": "double"},
                            {"name": "COMMUTE_TIME", "nullable": True, "metadata": {}, "type": "double"},
                            {"name": "AGE_BEGIN_PERIOD", "nullable": True, "metadata": {}, "type": "double"},
                            {"name": "GENDER_CODE", "nullable": True, "metadata": {}, "type": "double"},
                            {"name": "PERIOD_TOTAL_DAYS", "nullable": True, "metadata": {}, "type": "double"},
                            {"name": "STARTING_SALARY", "nullable": True, "metadata": {}, "type": "double"},
                            {"name": "ENDING_SALARY", "nullable": True, "metadata": {}, "type": "double"},
                            {"name": "NB_INCREASES", "nullable": True, "metadata": {}, "type": "double"},
                            {"name": "BONUS", "nullable": True, "metadata": {}, "type": "double"},
                            {"name": "NB_BONUS", "nullable": True, "metadata": {}, "type": "double"},
                            {"name": "VACATION_DAYS_TAKEN", "nullable": True, "metadata": {}, "type": "double"},
                            {"name": "SICK_DAYS_TAKEN", "nullable": True, "metadata": {}, "type": "double"},
                            {"name": "PROMOTIONS", "nullable": True, "metadata": {}, "type": "double"},
                            {"name": "NB_MANAGERS", "nullable": True, "metadata": {}, "type": "double"},
                            {"name": "DAYS_IN_POSITION", "nullable": True, "metadata": {}, "type": "double"},
                            {"name": "DAYS_SINCE_LAST_RAISE", "nullable": True, "metadata": {}, "type": "double"},
                            {"name": "RANKING_CODE", "nullable": True, "metadata": {}, "type": "double"},
                            {"name": "OVERTIME", "nullable": True, "metadata": {}, "type": "double"},
                            {"name": "DBLOVERTIME", "nullable": True, "metadata": {}, "type": "double"},
                            {"name": "TRAVEL", "nullable": True, "metadata": {}, "type": "double"}
                        ]
                    }
                }
            ]

The cell below will authenticate with the IBM FactSheet service using credentials you have already supplied and initialize FactSheet monitoring for this model. Note that Python notebooks in Watson Studio have full support for `pip install`, which allows you to add whatever libraries you need to the notebook environment. For example, if you wanted to use Python to parse command line arguments, you could run `!pip install argparse`.

If you receive an error related to the project access token, it is likely because you either did not insert the project access token as instructed at the beginning of the notebook, or did not run the cell after it was inserted. You will need to return to the beginning of the notebook, ensure the cell is inserted, and execute it.

In [9]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()
token = wslib.auth.get_current_token()

### Setup WML Client

In [17]:
from ibm_watson_machine_learning import APIClient
import os

token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
   "token": token,
   "instance_id" : "openshift",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "4.6"
}

wml_client = APIClient(wml_credentials)

### Setup FactSheet Client

In [39]:
# @hidden_cell
from ibm_aigov_facts_client import AIGovFactsClient,CloudPakforDataConfig
cpd_creds=CloudPakforDataConfig(service_url="https://internal-nginx-svc:12443",
    username="wilfried.hoge",
    api_key="F7fw7s8fifSfKWeE7iO2zt7DhB5duwf4MTzYWUOI")

CPD_URL=os.environ['RUNTIME_ENV_APSX_URL'][len('https://api.'):]
PROJECT_UID= os.environ['PROJECT_ID']
CONTAINER_ID=PROJECT_UID
CONTAINER_TYPE='project'
EXPERIMENT_NAME='predictive_attrition v2'

wml_client.set.default_project(PROJECT_UID)

'SUCCESS'

In [40]:
#Facts Client
facts_client = AIGovFactsClient(cloud_pak_for_data_configs=cpd_creds, 
                                experiment_name=EXPERIMENT_NAME,
                                container_type="project",
                                container_id=CONTAINER_ID)

2023/01/19 15:18:35 INFO : Experiment successfully created with ID 3 and name predictive_attrition v2
2023/01/19 15:18:36 INFO : Autolog enabled Successfully


## Define Software Specifications & Train Model

In [41]:
fields=X_train.columns.tolist()
metadata_dict = {'target_col' : 'ATTRITION', 'fields':fields}

In [42]:
import sklearn
sklearn.__version__

#https://dataplatform.cloud.ibm.com/docs/content/wsj/wmls/wmls-deploy-python-types.html

'1.1.1'

In [43]:
#Show Supported Frameworks
wml_client.software_specifications.list(10)

--------------------------  ------------------------------------  ----
NAME                        ASSET_ID                              TYPE
default_py3.6               0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
runtime-22.1-r3.6           018ebea5-1d1f-5fec-b93e-5e2ab30e7f38  base
kernel-spark3.2-scala2.12   020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt  069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6     09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12  09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9   0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6       0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                  0e6e79df-875e-4f24-8ae9-62dcc2148306  base
pytorch_1.1-py3.6           10ac12d6-6b30-4ccd-8392-3e922c096a92  base
--------------------------  ------------------------------------  ----


In [44]:
software_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-22.1-py3.9")
print("Software Specification ID: {}".format(software_spec_uid))



Software Specification ID: 12b83a17-24d8-5082-900f-0ab31fbfd3cb


In [45]:

metadata = {
            wml_client.repository.ModelMetaNames.NAME: 'Random Forrest',
            wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0',
            wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
}

facts_client.export_facts.prepare_model_meta(wml_client=wml_client,meta_props=metadata)

{'name': 'Random Forrest',
 'type': 'scikit-learn_1.0',
 'software_spec': '12b83a17-24d8-5082-900f-0ab31fbfd3cb',
 'custom': {'experiment_id': '979e2bd5963a4950a86fe86546a7c863',
  'experiment_name': 'predictive_attrition v2'}}

In [51]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf = RandomForestClassifier(n_estimators=50)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

2023/01/19 15:49:44 INFO : logging results to factsheet for run_id 8688abff1cf244a1a4f1624306f5c212
2023/01/19 15:49:49 ERROR : Something went wrong when publishing to factsheet. ERROR: 400.{"code":"Bad Request","message":"CATSV5025E: Unable to find Catalog for project 'a805717d-0b8a-43ae-a461-518ee76894c6' while executing POST https://internal-nginx-svc.zen-46.svc:12443/v2/asset_types/notebook_experiment/search?project_id=a805717d-0b8a-43ae-a461-518ee76894c6","trace":"2zgpbj5uue2yjq0rjyu8flbeb"}


In [47]:
from sklearn import metrics
import pandas as pd
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8611111111111112


In [48]:
feature_imp = pd.Series(clf.feature_importances_,index=X.columns).sort_values(ascending=False)
feature_imp

COMMUTE_TIME             0.205319
OVERTIME                 0.157364
DBLOVERTIME              0.102783
DAYS_IN_POSITION         0.066943
DAYS_SINCE_LAST_RAISE    0.055362
PERIOD_TOTAL_DAYS        0.054036
DAYS_WITH_COMPANY        0.052268
ENDING_SALARY            0.044760
STARTING_SALARY          0.041188
AGE_BEGIN_PERIOD         0.038191
VACATION_DAYS_TAKEN      0.035396
POSITION_CODE            0.033182
SICK_DAYS_TAKEN          0.033067
RANKING_CODE             0.024940
BONUS                    0.023298
DEPARTMENT_CODE          0.018098
GENDER_CODE              0.005414
NB_BONUS                 0.004205
NB_INCREASES             0.004188
NB_MANAGERS              0.000000
PROMOTIONS               0.000000
TRAVEL                   0.000000
dtype: float64

In [49]:
facts_client.runs.list_runs_by_experiment('1')

,run_id,experiment_id,published,artifact_uri,start_time,end_time


The next three cells fit the data the the model using a Random Forest classifier, run predictions on the test data, and then print out the accuracy for how the model did on the test data. Finally, the notebook calculates and displays feature importance. For more information on Random Forest classifiers, see [here](https://www.ibm.com/cloud/learn/random-forest).

The next three cells export data from the model you just created to the FactSheet. The first lists experiments tracked by FactSheets. The second writes the URL and other info on this notebook as custom data to the FactSheet. Note that any data can be written to the FactSheet that might be helpful for model validators.

In [50]:
nb_name = "attrition model creation and deployment"
nb_asset_id = "tbd"
nb_asset_url = "https://" + CPD_URL + "/analytics/notebooks/v2/" + nb_asset_id + "?projectid=" + PROJECT_UID + "&context=cpdaas"

latestRunId = facts_client.runs.list_runs_by_experiment('1').sort_values('start_time').iloc[-1]['run_id']
facts_client.runs.set_tags(latestRunId, {"Notebook name": nb_name, "Notebook id": nb_asset_id, "Notebook URL" : nb_asset_url})
facts_client.export_facts.export_payload(latestRunId)

IndexError: single positional indexer is out-of-bounds

In [ ]:
RUN_ID=facts_client.runs.get_current_run_id()
facts_client.export_facts.export_payload(RUN_ID)

Finally, the model is stored to the project with all of the metadata defined above.

In [ ]:
print("Storing model...")
published_model_details = wml_client.repository.store_model(
    model=clf, 
    meta_props=metadata,
    training_target=['ATTRITION'],
    training_data=X)
model_uid = wml_client.repository.get_model_id(published_model_details)

print("Done")
print("Model ID: {}".format(model_uid))

# Spark Model

Next, the notebook uses Apache Spark to create a second model. Because you specified a Spark environment when you created this notebook, the `pyspark` runtime will be available without needing to be installed via `pip`.

In [ ]:
try:
    from pyspark.sql import SparkSession
except:
    print('Error: Spark runtime is missing. If you are using Watson Studio change the notebook runtime to Spark by clicking \
    the Vew notebook info button above (the lowercase i in a circle). Click on the Environment tab and use the Environment \
    definition dropdown to select an environment with Spark and Python.')
    raise
spark.version

In [ ]:
import itc_utils.flight_service as itcfs

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

readClient = itcfs.get_flight_client()

nb_data_request = {
    'data_name': """modeling_records_2022.csv""",
    'interaction_properties': {
        #'row_limit': 500,
        'infer_schema': 'true',
        'infer_as_varchar': 'false'
    }
}

flightInfo = itcfs.get_flight_info(readClient, nb_data_request=nb_data_request)

df_data_2 = itcfs.read_pandas_and_concat(readClient, flightInfo, timeout=240)
df_data_2 = spark.createDataFrame(df_data_2)
df_data_2.show(10)


Similar to the `sklearn` model, you need to specify metadata for the spark model.

In [ ]:
software_spec_uid = wml_client.software_specifications.get_id_by_name('spark-mllib_3.3')
print("Software Specification ID: {}".format(software_spec_uid))

meta_props1 = {
            wml_client.repository.ModelMetaNames.NAME: 'Attrition Challenger – Spark',
            wml_client.repository.ModelMetaNames.TYPE: 'mllib_3.3',
            wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
}

facts_client.export_facts.prepare_model_meta(wml_client=wml_client,meta_props=meta_props1)

software_spec_uid = wml_client.software_specifications.get_id_by_name("spark-mllib_3.3")
print("Software Specification ID: {}".format(software_spec_uid))


model_props = {
    wml_client._models.ConfigurationMetaNames.NAME:"{}".format("attrition challenger - spark"),
    wml_client._models.ConfigurationMetaNames.TYPE: "mllib_3.3",
    wml_client._models.ConfigurationMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    wml_client._models.ConfigurationMetaNames.TRAINING_DATA_REFERENCES: training_data_references,
    wml_client._models.ConfigurationMetaNames.LABEL_FIELD: "ATTRITION"
}

facts_client.export_facts.prepare_model_meta(wml_client=wml_client,meta_props=model_props)

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model

For the second model, you will create a Gradient Boosted Tree classifier. For more information on Gradient Boosting, see [here](https://www.ibm.com/cloud/learn/boosting).

In [ ]:
from pyspark.sql.types import FloatType
for field in fields:
    df_data_2=df_data_2.withColumn(field,df_data_2[field].cast("float").alias(field))
df_data_2=df_data_2.withColumn('ATTRITION',df_data_2['ATTRITION'].cast("int").alias('ATTRITTION'))
df_data_2.take(5)

In [ ]:
va = VectorAssembler(inputCols = fields, outputCol='features')
va_df = va.transform(df_data_2)
va_df = va_df.select(['features', 'ATTRITION'])
va_df.show(3)

In [ ]:
gbtc = GBTClassifier(labelCol="ATTRITION", maxIter=20)

pipeline = Pipeline(stages=[va, gbtc])

In [ ]:
split_data = df_data_2.randomSplit([0.8, 0.2], 24)
train_data = split_data[0]
test_data = split_data[1]

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))

In [ ]:
spark_model = pipeline.fit(train_data)

pred = spark_model.transform(test_data)
pred.show(3) 

In [ ]:
evaluator = BinaryClassificationEvaluator()
evaluator.setLabelCol("ATTRITION")
print("Test Area Under ROC: " + str(evaluator.evaluate(pred, {evaluator.metricName: "areaUnderROC"})))

In [ ]:
#Specify ML Model and Save it into WML
sofware_spec_uid = wml_client.software_specifications.get_id_by_name("spark-mllib_3.3")




metadata = {
            wml_client.repository.ModelMetaNames.NAME: 'Spark GBT',
            wml_client.repository.ModelMetaNames.TYPE: 'mllib_3.3',
            wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

published_model = wml_client.repository.store_model(
    model=spark_model,
    meta_props=meta_props1,
    training_target=['ATTRITION'],
    training_data=train_data,
    pipeline=pipeline
)

In [ ]:
print("Storing model...")
published_model_details = wml_client.repository.store_model(
    model=clf, 
    meta_props=metadata,
    training_target=['ATTRITION'],
    training_data=X)
model_uid = wml_client.repository.get_model_id(published_model_details)

print("Done")
print("Model ID: {}".format(model_uid))

In [ ]:
print("Storing spark model...")
published_model_details = wml_client.repository.store_model(
    model=spark_model, 
    meta_props=meta_props1,
    training_target=['ATTRITION'],
    training_data=train_data,
    pipeline=pipeline
)
model_uid = wml_client.repository.get_model_id(published_model_details)

print("Done")
print("Model ID: {}".format(model_uid))


# Congratulations!

You have completed this notebook. You can now return to the [Data and AI Live Demos lab page](https://cp4d-outcomes.techzone.ibm.com/data-fabric-lab/trusted-ai) and continue with the lab.